In [3]:
from transformers import DPTForDepthEstimation
from safetensors.torch import load_file
import torch

In [11]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [5]:
model_path = "/home/coder/low-power-cv-challenge-track3/outputs/checkpoint-7000/"
model = DPTForDepthEstimation.from_pretrained(model_path)
state_dict = load_file(f"{model_path}/model.safetensors")
new_state_dict = {k.replace("module.model.", ""): v for k, v in state_dict.items()}  # Remove the prefix
model.load_state_dict(new_state_dict) 
model.to("cuda")
model.eval()


Some weights of the model checkpoint at /home/coder/low-power-cv-challenge-track3/outputs/checkpoint-7000/ were not used when initializing DPTForDepthEstimation: ['module.model.backbone.embeddings.cls_token', 'module.model.backbone.embeddings.mask_token', 'module.model.backbone.embeddings.patch_embeddings.projection.bias', 'module.model.backbone.embeddings.patch_embeddings.projection.weight', 'module.model.backbone.embeddings.position_embeddings', 'module.model.backbone.encoder.layer.0.attention.attention.key.bias', 'module.model.backbone.encoder.layer.0.attention.attention.key.weight', 'module.model.backbone.encoder.layer.0.attention.attention.query.bias', 'module.model.backbone.encoder.layer.0.attention.attention.query.weight', 'module.model.backbone.encoder.layer.0.attention.attention.value.bias', 'module.model.backbone.encoder.layer.0.attention.attention.value.weight', 'module.model.backbone.encoder.layer.0.attention.output.dense.bias', 'module.model.backbone.encoder.layer.0.attent

DPTForDepthEstimation(
  (backbone): Dinov2Backbone(
    (embeddings): Dinov2Embeddings(
      (patch_embeddings): Dinov2PatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Dinov2Encoder(
      (layer): ModuleList(
        (0-11): 12 x Dinov2Layer(
          (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
          (attention): Dinov2SdpaAttention(
            (attention): Dinov2SdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): Dinov2SelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
 

In [6]:
print(model.get_memory_footprint())

148790404


In [13]:
from torch.ao.quantization import quantize_dynamic
model = quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)


In [14]:
print(model.get_memory_footprint())

58965124


In [15]:
58965124/148790404

0.3962965514899738

In [18]:
import os
from PIL import Image
from typing import Optional, Tuple, Dict
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class IndoorOutdoorDataset(Dataset):
    """
    Custom dataset for loading indoor and outdoor images along with their labels.
    Expected folder structure:
    
        root_dir/
            indoor/              # .jpg images for indoor scenes
            indoor_labels/       # .pt label tensors (shape: 384x384) matching indoor images
            outdoor/             # .jpg images for outdoor scenes
            outdoor_labels/      # .pt label tensors (shape: 384x384) matching outdoor images
    """
    def __init__(self, 
                 root_dir: str, 
                 transform: Optional[transforms.Compose] = None, 
                 image_size: Tuple[int, int] = (384, 384)):
        """
        Args:
            root_dir (str): Path to the dataset directory.
            transform (torchvision.transforms.Compose, optional): Transformations to apply to images.
            image_size (tuple): Desired image size (height, width).
        """
        self.root_dir = root_dir
        # Use a default transform if none is provided.
        self.transform = transform if transform is not None else transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor()
        ])
        self.image_size = image_size
        
        # List to store tuples of (image_path, label_path)
        self.data = []
        
        # Process indoor images and their labels.
        indoor_dir = os.path.join(root_dir, "indoor")
        indoor_labels_dir = os.path.join(root_dir, "indoor_labels")
        if os.path.exists(indoor_dir) and os.path.exists(indoor_labels_dir):
            for img_name in os.listdir(indoor_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(indoor_dir, img_name)
                    # Construct the label file name by replacing the image extension with .pt
                    label_name = os.path.splitext(img_name)[0] + ".pt"
                    label_path = os.path.join(indoor_labels_dir, label_name)
                    if os.path.exists(label_path):
                        self.data.append((img_path, label_path))
        
        # Process outdoor images and their labels.
        outdoor_dir = os.path.join(root_dir, "outdoor")
        outdoor_labels_dir = os.path.join(root_dir, "outdoor_labels")
        if os.path.exists(outdoor_dir) and os.path.exists(outdoor_labels_dir):
            for img_name in os.listdir(outdoor_dir):
                if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(outdoor_dir, img_name)
                    label_name = os.path.splitext(img_name)[0] + ".pt"
                    label_path = os.path.join(outdoor_labels_dir, label_name)
                    if os.path.exists(label_path):
                        self.data.append((img_path, label_path))
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        img_path, label_path = self.data[idx]
        
        # Load the image and convert it to RGB.
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        # Load the label tensor.
        label = torch.load(label_path,map_location=torch.device('cpu'))
        
        # Return dictionary uses key "label" to match your DataLoader usage.
        return {"pixel_values": image, "label": label}

def create_data_loader(
    data_dir: str,
    batch_size: int = 16,
    image_size: Tuple[int, int] = (384, 384),
    num_workers: int = 4,
    transform: Optional[transforms.Compose] = None,
) -> DataLoader:
    """
    Create a DataLoader for the combined indoor-outdoor dataset.
    
    Args:
        data_dir (str): Path to the data directory containing 'indoor', 'indoor_labels', 'outdoor', and 'outdoor_labels' subfolders.
        batch_size (int): Batch size for the DataLoader.
        image_size (tuple): Desired image size for resizing.
        num_workers (int): Number of worker processes for data loading.
        transform (torchvision.transforms.Compose, optional): Custom transformations for images.
        
    Returns:
        DataLoader: DataLoader for the dataset.
    """
    dataset = IndoorOutdoorDataset(
        root_dir=data_dir,
        transform=transform,
        image_size=image_size
    )
    
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )
    
    return data_loader


In [19]:
data_loader = create_data_loader(data_dir="/home/coder/low-power-cv-challenge-track3/data/eval",batch_size=1,image_size=(384, 384),num_workers=4)


In [20]:
for data in data_loader:
    input_data = data['pixel_values']
    print(input_data.shape)
    target = data['label']
    pred = model(input_data.to("cuda"))
    print(pred.predicted_depth.shape)
    break

torch.Size([1, 3, 384, 384])
torch.Size([1, 448, 448])


In [10]:
pred.predicted_depth.shape

torch.Size([1, 448, 448])